In [ ]:
# Install dependencies
!pip install -U pip
!pip install ultralytics
!pip install matplotlib opencv-python-headless pandas tqdm

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
# Quick check for dataset
import glob
print("Train images:", len(glob.glob("/content/drive/MyDrive/space_station_safety/dataset/train/images/*")))
print("Val images:", len(glob.glob("/content/drive/MyDrive/space_station_safety/dataset/val/images/*")))


In [ ]:
# Create data/data.yaml file
import os
yaml_content = """
train: /content/drive/MyDrive/space_station_safety/dataset/train/images
val:   /content/drive/MyDrive/space_station_safety/dataset/val/images

nc: 7
names: ['helmet', 'fire_extinguisher', 'first_aid', 'oxygen_mask', 'toolbox', 'gas_leak_detector', 'exit_sign']
"""
os.makedirs("data", exist_ok=True)
with open("data/data.yaml","w") as f:
    f.write(yaml_content)
print(open("data/data.yaml").read())

In [ ]:
# Train baseline (yolov8n)
!yolo train model=yolov8n.pt data=data/data.yaml epochs=20 imgsz=640 batch=16 device=0

In [ ]:
# Save weights to Drive
!mkdir -p /content/drive/MyDrive/space_station_safety/artifacts
!cp runs/detect/train/weights/best.pt /content/drive/MyDrive/space_station_safety/artifacts/best.pt
!ls -lh /content/drive/MyDrive/space_station_safety/artifacts

In [ ]:
# Validate and save plots
!yolo val model=/content/drive/MyDrive/space_station_safety/artifacts/best.pt data=data/data.yaml plots=True
!cp -r runs/detect/val /content/drive/MyDrive/space_station_safety/val_results

In [ ]:
# Predict on validation images
!yolo predict model=/content/drive/MyDrive/space_station_safety/artifacts/best.pt source=/content/drive/MyDrive/space_station_safety/dataset/val/images save=True conf=0.25
!mkdir -p /content/drive/MyDrive/space_station_safety/predictions
!cp -r runs/detect/predict/* /content/drive/MyDrive/space_station_safety/predictions

In [ ]:
# Export model to ONNX
from ultralytics import YOLO
model = YOLO("/content/drive/MyDrive/space_station_safety/artifacts/best.pt")
model.export(format="onnx")
!cp best.onnx /content/drive/MyDrive/space_station_safety/artifacts/